We are finding that June and July and sometimes May are consistently the best months for a theatrical release. December and to a lesser extend, November and January. Then February, March, and April are dead months. Lastly August, September, and October are ok.

First, I will take the "Earliest Release Data" column and parse out into "country of earliest release" and "Earliest Release DateTime"
Then I will use the month of the date time object to split into 1 of 4 categories as discussed above.
Then I will compare the datetime to the datetimes of federal holidays and give a 1 if near the date time of a federal holiday, 0 otherwise, were near is defined in parametes

In [65]:
from datetime import datetime
import pandas as pd


In [66]:
test = "January 31, 2014 (Spain)"

In [67]:
def to_dt(release_date):
    try:
        date_l = release_date.split("(")[0]
        # Sample string representing a date
        date_string = date_l.strip()

        # Convert the string to a datetime object
        datetime_object = datetime.strptime(date_string, '%B %d, %Y')
        return datetime_object
    except:
        return pd.NaT

In [68]:
df = pd.read_csv("./data_with_encoded_mpaa.csv")
df['US Release Date'] = pd.to_datetime(df['US Release Date'], errors='coerce')
for date in df["US Release Date"]:
    print(date)

2013-09-25 00:00:00+00:00
2013-01-18 00:00:00+00:00
2013-04-05 00:00:00+00:00
2013-02-22 00:00:00+00:00
2013-02-08 00:00:00+00:00
2013-02-01 00:00:00+00:00
2013-11-09 00:00:00+00:00
2013-08-30 00:00:00+00:00
2013-12-20 00:00:00+00:00
2013-01-18 00:00:00+00:00
2013-06-14 00:00:00+00:00
2013-08-16 00:00:00+00:00
2013-06-20 00:00:00+00:00
2013-07-31 00:00:00+00:00
2014-07-11 00:00:00+00:00
2014-02-07 00:00:00+00:00
2013-04-23 00:00:00+00:00
2013-02-19 00:00:00+00:00
2013-10-04 00:00:00+00:00
2013-01-22 00:00:00+00:00
2013-10-25 00:00:00+00:00
2013-12-25 00:00:00+00:00
2013-02-14 00:00:00+00:00
2013-04-04 00:00:00+00:00
2013-10-18 00:00:00+00:00
2013-08-23 00:00:00+00:00
2013-10-17 00:00:00+00:00
2014-01-24 00:00:00+00:00
2014-04-17 00:00:00+00:00
2014-03-14 00:00:00+00:00
2014-01-04 00:00:00+00:00
2013-11-22 00:00:00+00:00
2015-02-06 00:00:00+00:00
2013-05-28 00:00:00+00:00
2013-04-03 00:00:00+00:00
2013-12-13 00:00:00+00:00
2013-07-22 00:00:00+00:00
2013-05-03 00:00:00+00:00
2013-08-09 0

In [69]:
df

,Genres,IMDB Title,MPAA,Domestic Distributor,Domestic Opening,Earliest Release Date,IMDB Budget,TMDB Budget,Genre ID,Genre Name,...,MPAA_TMDB,Domestic Distributor ID,Domestic Distributor Score,Merged_MPAA,G,NC-17,NR,PG,PG-13,R
0,Documentary Sport,Into the Mind (2013),NaN,NaN,NaN,"January 31, 2014 (Spain)",0,0,[99],['Documentary'],...,NaN,0,0,NR,0,0,1,0,0,0
1,Action Crime Drama Mystery Thriller,Broken City (2013),R,Twentieth Century Fox,8268908.0,"January 18, 2013 (Domestic, EMEA)",35000000,56000000,"[53, 80, 18]","['Thriller', 'Crime', 'Drama']",...,R,1,1751,R,0,0,0,0,0,1
2,Drama,Redwood Highway (2013),PG-13,Monterey Media,30.0,"April 11, 2014 (Domestic)",0,0,[18],['Drama'],...,PG-13,2,0,PG-13,0,0,0,0,1,0
3,Biography Drama History War,"Bless Me, Ultima (2012)",PG-13,Arenas Entertainment,470109.0,"February 22, 2013 (Domestic)",0,0,"[18, 10752]","['Drama', 'War']",...,PG-13,3,0,PG-13,0,0,0,0,1,0
4,Action Adventure Comedy Horror Sci-Fi Thriller,Spiders (2013),PG-13,Millennium Entertainment,NaN,"February 8, 2013 (Domestic)",0,7000000,"[878, 53]","['Science Fiction', 'Thriller']",...,PG-13,4,0,PG-13,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3590,Action Comedy Crime,Dancing Ninja (2010),NaN,NaN,NaN,"July 19, 2012 (United Arab Emirates)",0,0,"[28, 35, 80]","['Action', 'Comedy', 'Crime']",...,NR,0,0,NR,0,0,1,0,0,0
3591,Biography Documentary Music,Ain't in It for My Health: A Film About Levon ...,NaN,Kino Lorber,7937.0,"April 19, 2013 (Domestic)",0,0,"[99, 10402]","['Documentary', 'Music']",...,NR,29,0,NR,0,0,1,0,0,0
3592,Biography Drama,Leonie (2010),PG-13,NaN,NaN,"November 20, 2010 (Japan)",0,0,[18],['Drama'],...,PG-13,0,0,PG-13,0,0,0,0,1,0
3593,Drama Family,Ways to Live Forever (2010),PG-13,NaN,NaN,"October 29, 2010 (Spain)",0,0,"[18, 10751]","['Drama', 'Family']",...,PG-13,0,0,PG-13,0,0,0,0,1,0


In [70]:
def split_by_month(date):
    try:
        if date.month in {5,6,7}:
            return "MJJ_3"
        elif date.month in {8,9,10}:
            return "ASO_4"
        elif date.month in {11,12,1}:
            return "NDJ_1"
        elif date.month in {2,3,4}:
            return "FMA_2"
    except:
        return pd.NA

In [71]:
df["Season"] = df.apply(lambda x: split_by_month(x["US Release Date"]), axis=1)

In [72]:
df["Season"]

0       ASO_4
1       NDJ_1
2       FMA_2
3       FMA_2
4       FMA_2
        ...  
3590    ASO_4
3591    FMA_2
3592    FMA_2
3593    MJJ_3
3594    MJJ_3
Name: Season, Length: 3595, dtype: object

In [73]:
from datetime import datetime, timedelta, date
import holidays
def replace_date_with_datetime(input_dict):
    """
    Replace keys that are datetime.date objects with datetime.datetime objects.

    Parameters:
    input_dict (dict): The dictionary with datetime.date keys.

    Returns:
    dict: A new dictionary with datetime.datetime keys.
    """
    new_dict = {}
    for key, value in input_dict.items():
        # Check if the key is a datetime.date object and not a datetime object
        if isinstance(key, date) and not isinstance(key, datetime):
            # Convert to datetime.datetime
            new_key = datetime(key.year, key.month, key.day)
        else:
            # Keep the key as it is
            new_key = key
        new_dict[new_key] = value

    return new_dict

def is_near_holiday(input_date, country='US', days_before=7, days_after=5):
    """
    Check if a given date is within a week prior to or 5 days after a US holiday.

    Parameters:
    input_date (datetime): The date to check.
    country (str): The country code for the holidays. Default is 'US'.

    Returns:
    bool: True if the date is near a holiday, False otherwise.
    """
    us_holidays = holidays.CountryHoliday(country, years= [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023])
    us_holidays = replace_date_with_datetime(us_holidays)
    # Check a week before and 5 days after each holiday in the year of the input date
    for date in us_holidays.keys():
        if input_date.timestamp() >= (date - timedelta(days=days_before)).timestamp() and input_date.timestamp() <= (date + timedelta(days=days_after)).timestamp():
            return True

    return False

# Example usage
example_date = datetime(2020, 12, 25)  # New Year's Day in 2023
print(is_near_holiday(example_date))


True


In [74]:
df["Holiday"] = df.apply(lambda x: is_near_holiday(x["US Release Date"]), axis=1)
df["Holiday"]

0       False
1        True
2       False
3        True
4       False
        ...  
3590     True
3591    False
3592    False
3593    False
3594    False
Name: Holiday, Length: 3595, dtype: bool

In [75]:
df[df["Holiday"] == True]

,Genres,IMDB Title,MPAA,Domestic Distributor,Domestic Opening,Earliest Release Date,IMDB Budget,TMDB Budget,Genre ID,Genre Name,...,Domestic Distributor Score,Merged_MPAA,G,NC-17,NR,PG,PG-13,R,Season,Holiday
1,Action Crime Drama Mystery Thriller,Broken City (2013),R,Twentieth Century Fox,8268908.0,"January 18, 2013 (Domestic, EMEA)",35000000,56000000,"[53, 80, 18]","['Thriller', 'Crime', 'Drama']",...,1751,R,0,0,0,0,0,1,NDJ_1,True
3,Biography Drama History War,"Bless Me, Ultima (2012)",PG-13,Arenas Entertainment,470109.0,"February 22, 2013 (Domestic)",0,0,"[18, 10752]","['Drama', 'War']",...,0,PG-13,0,0,0,0,1,0,FMA_2,True
6,Action Crime Drama Thriller,Out of the Furnace (2013),R,Relativity Media,5220288.0,"December 4, 2013 (Domestic)",22000000,22000000,"[53, 18, 80]","['Thriller', 'Drama', 'Crime']",...,241,R,0,0,0,0,0,1,NDJ_1,True
7,Documentary Music,One Direction: This Is Us (2013),PG,TriStar Pictures,15815497.0,"August 28, 2013 (EMEA)",10000000,10000000,"[99, 10402]","['Documentary', 'Music']",...,114,PG,0,0,0,1,0,0,ASO_4,True
8,Action Adventure Animation Drama Family,Tarzan (2013),PG,NaN,NaN,"December 12, 2013 (United Arab Emirates)",0,0,"[16, 28, 12, 10751]","['Animation', 'Action', 'Adventure', 'Family']",...,0,PG,0,0,0,1,0,0,NDJ_1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3553,Adventure Comedy Horror,Dead Before Dawn 3D (2012),R,Gaiam Vivendi Entertainment,NaN,"October 11, 2012 (Russia/CIS)",0,0,"[12, 35, 27]","['Adventure', 'Comedy', 'Horror']",...,0,R,0,0,0,0,0,1,ASO_4,True
3578,Comedy Horror,100 Bloody Acres (2012),NaN,Doppelganger Releasing,3419.0,"June 28, 2013 (Domestic)",0,0,"[35, 27]","['Comedy', 'Horror']",...,0,R,0,0,0,0,0,1,MJJ_3,True
3580,Action Adventure Animation Sci-Fi,War of the Worlds: Goliath (2012),PG-13,Barking Cow Media Group,13385.0,"March 7, 2014 (Domestic)",0,0,"[12, 28, 16, 878]","['Adventure', 'Action', 'Animation', 'Science ...",...,0,PG-13,0,0,0,0,1,0,MJJ_3,True
3585,Biography Documentary Music,Crossfire Hurricane (2012),NaN,NaN,NaN,"November 29, 2012 (New Zealand)",0,0,"[99, 10402]","['Documentary', 'Music']",...,0,NR,0,0,1,0,0,0,MJJ_3,True


In [76]:
for col in df.columns:
    print(col)

Genres
IMDB Title
MPAA
Domestic Distributor
Domestic Opening
Earliest Release Date
IMDB Budget
TMDB Budget
Genre ID
Genre Name
IMDB ID
Production Company ID
Production Company Name
Release Date
TMDB Revenue
Runtime
TMDB Title
TMDB ID
cast_ids
order
director_ids
IMDB Domestic Revenue
international_revenue
worldwide_revenue
Merged Budget
Merged Revenue
US Release Date
Release Year
Raw Star Scores
Unweighted Star Score
Simple Weight Star Score
Log Weight Star Score
Exponential Weight Star Score
Movie Contribution to Director and Production Scores
Total Director Score
Avg Director Score
Total Production Company Score
Avg Production Company Score
Genre Cluster
MPAA_TMDB
Domestic Distributor ID
Domestic Distributor Score
Merged_MPAA
G
NC-17
NR
PG
PG-13
R
Season
Holiday


In [77]:
import cpi
import numpy as np

def inflate(date, dollars):
    try:
        if date.year == 2023:
            multiplier = 100/302.9
            return dollars * multiplier
        else:
            return cpi.inflate(dollars, date.year, to=1983) #when cpi was 100
    except:
        return np.nan


In [78]:
df["Adj Merged Budget"] = df.apply(lambda x: inflate(x["US Release Date"], x["Merged Budget"]), axis=1)
df["Adj Merged Revenue"] = df.apply(lambda x: inflate(x["US Release Date"], x["Merged Revenue"]), axis=1)
df[["IMDB ID", "Adj Merged Budget", "Merged Budget"]]

,IMDB ID,Adj Merged Budget,Merged Budget
0,tt2673812,4.275467e+04,100000
1,tt1235522,1.496414e+07,35000000
2,tt2515164,4.275467e+04,100000
3,tt1390398,4.275467e+04,100000
4,tt1659216,2.992827e+06,7000000
...,...,...,...
3590,tt1294141,4.275467e+04,100000
3591,tt1362518,4.275467e+04,100000
3592,tt1426328,4.275467e+04,100000
3593,tt1446208,4.275467e+04,100000


In [79]:
df.to_csv("data_with_holiday_and_cpi.csv")